<a href="https://colab.research.google.com/github/lamkaiyi/RAG_with_Llama/blob/main/alternative_evaluation_methods.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip -q install sentence-transformers bert-score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 21.8 MB/s eta 0:00:00


In [ ]:
#mount Google Drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Import the Questions, Answers and Contexts

In [ ]:
import pandas as pd

samples = pd.read_csv('drive/MyDrive/results/samples.txt', sep='+')

In [ ]:
samples

,Question,Answer,Context
0,Provide an example of a situation where applyi...,The kernel trick can be applied when dealing w...,"[""Support vector machines have attracted much ..."
1,Discuss the challenges posed by imbalanced dat...,Imbalanced datasets pose significant challenge...,"[""Application of fuzzy support vector machine ..."
2,Explain strategies to mitigate the impact of c...,Researchers have proposed various techniques t...,"[""Application of fuzzy support vector machine ..."
3,What are the differences between support vecto...,Support vector machines (SVMs) differ from oth...,"['In this paper, we study the support vector m..."
4,In what scenarios are support vector machines ...,"Based on the text, support vector machines may...","[""Support vector machines have attracted much ..."


# Use Vectara model
The first document is the source document, and the second document is validated against the first for factual consistency, e.g. as a summary of the first or a claim drawn from the source.

In [ ]:
from sentence_transformers import CrossEncoder
model = CrossEncoder('vectara/hallucination_evaluation_model')
model
scores = model.predict([
    ["A man walks into a bar and buys a drink", "A bloke swigs alcohol at a pub"],
    ["A person on a horse jumps over a broken down airplane.", "A person is at a diner, ordering an omelette."],
    ["A person on a horse jumps over a broken down airplane.", "A person is outdoors, on a horse."],
    ["A boy is jumping on skateboard in the middle of a red bridge.", "The boy skates down the sidewalk on a blue bridge"],
    ["A man with blond-hair, and a brown shirt drinking out of a public water fountain.", "A blond drinking water in public."],
    ["A man with blond-hair, and a brown shirt drinking out of a public water fountain.", "A blond man wearing a brown shirt is reading a book."],
    ["Mark Wahlberg was a fan of Manny.", "Manny was a fan of Mark Wahlberg."],
])

scores

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.02k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/738M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/575 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/8.65M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/23.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

array([6.1051559e-01, 4.7493709e-04, 9.9639291e-01, 2.1221573e-04,
       9.9599433e-01, 1.4127002e-03, 2.8262993e-03], dtype=float32)

In [ ]:
model.predict([
    samples.loc[0]['Context'], samples.loc[0]['Answer']
])

0.70251644

In [ ]:
vectara_score = samples.copy()
vectara_score['Vectara Hallucination Score'] = vectara_score.apply(
    lambda x: model.predict([x['Context'], x['Answer']]), #Evaluate the generated answer against the sources (Retrieved Context)
    axis=1
)


In [ ]:
vectara_score

,Question,Answer,Context,Vectara Hallucination Score
0,Provide an example of a situation where applyi...,The kernel trick can be applied when dealing w...,"[""Support vector machines have attracted much ...",0.702516
1,Discuss the challenges posed by imbalanced dat...,Imbalanced datasets pose significant challenge...,"[""Application of fuzzy support vector machine ...",0.883943
2,Explain strategies to mitigate the impact of c...,Researchers have proposed various techniques t...,"[""Application of fuzzy support vector machine ...",0.880702
3,What are the differences between support vecto...,Support vector machines (SVMs) differ from oth...,"['In this paper, we study the support vector m...",0.679304
4,In what scenarios are support vector machines ...,"Based on the text, support vector machines may...","[""Support vector machines have attracted much ...",0.758595


This model was trained using SentenceTransformers Cross-Encoder class. The model outputs a probabilitity from 0 to 1, 0 being a hallucination and 1 being factually consistent.



# Using BERT-Score

Note: might need to re-train the model weights before using for evaluation

In [ ]:

from bert_score import score

generated = ["The kernel trick can be applied when dealing with high-dimensional data or complex models. For instance, in image recognition tasks, the feature space can be very large, making it difficult to compute pairwise similarities between images."]
context = ["Support vector machines have attracted much attention in theoretical and in\napplied statistics. Main topics of recent interest are consistency, learning\nrates and robustness. In this article, it is shown that support vector machines\nare qualitatively robust. Since support vector machines can be represented by a\nfunctional on the set of all probability measures, qualitative robustness is\nproven by showing that this functional is continuous with respect to the\ntopology generated by weak convergence of probability measures."]

P, R, F1 = score(generated, context, lang='en', verbose=True)



config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

...

calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 1.35 seconds, 0.74 sentences/sec


In [ ]:
score(generated, context, lang='en', verbose=True)

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 1.22 seconds, 0.82 sentences/sec


(tensor([0.8486]), tensor([0.8164]), tensor([0.8321]))

In [ ]:
# Print scores
print(f"Precision: {P.mean()}")
print(f"Recall: {R.mean()}")
print(f"F1 Score: {F1.mean()}")

Precision: 0.8485579490661621
Recall: 0.8163638114929199
F1 Score: 0.8321495652198792


In [ ]:
df = samples.copy()
df

,Question,Answer,Context
0,Provide an example of a situation where applyi...,The kernel trick can be applied when dealing w...,"[""Support vector machines have attracted much ..."
1,Discuss the challenges posed by imbalanced dat...,Imbalanced datasets pose significant challenge...,"[""Application of fuzzy support vector machine ..."
2,Explain strategies to mitigate the impact of c...,Researchers have proposed various techniques t...,"[""Application of fuzzy support vector machine ..."
3,What are the differences between support vecto...,Support vector machines (SVMs) differ from oth...,"['In this paper, we study the support vector m..."
4,In what scenarios are support vector machines ...,"Based on the text, support vector machines may...","[""Support vector machines have attracted much ..."


In [ ]:
df['Precision_Recall_F1'] = df.apply(
    lambda x: score(cands=[x['Answer']], refs=[x['Context']], lang='en'),
    axis=1
)

In [ ]:

df

,Question,Answer,Context,Precision_Recall_F1
0,Provide an example of a situation where applyi...,The kernel trick can be applied when dealing w...,"[""Support vector machines have attracted much ...","([tensor(0.8324)], [tensor(0.7599)], [tensor(0..."
1,Discuss the challenges posed by imbalanced dat...,Imbalanced datasets pose significant challenge...,"[""Application of fuzzy support vector machine ...","([tensor(0.8131)], [tensor(0.7632)], [tensor(0..."
2,Explain strategies to mitigate the impact of c...,Researchers have proposed various techniques t...,"[""Application of fuzzy support vector machine ...","([tensor(0.8180)], [tensor(0.7468)], [tensor(0..."
3,What are the differences between support vecto...,Support vector machines (SVMs) differ from oth...,"['In this paper, we study the support vector m...","([tensor(0.8187)], [tensor(0.7542)], [tensor(0..."
4,In what scenarios are support vector machines ...,"Based on the text, support vector machines may...","[""Support vector machines have attracted much ...","([tensor(0.8442)], [tensor(0.7749)], [tensor(0..."


High Scores, even though from human understanding, the Answers do not draw from the retrieved Contexts.

---------